# Preprocesamiento de Datos
## Predicción de Clientes Fidelizables en E-commerce

### Objetivos del Preprocesamiento

Este notebook aborda la limpieza y preparación del dataset Online Retail para el análisis posterior. Los objetivos específicos incluyen:

1. **Tratamiento de valores faltantes**
   - Análisis del impacto de la eliminación de registros
   - Estrategias de imputación cuando sea apropiado
   - Justificación de decisiones de limpieza

2. **Eliminación de outliers y anomalías**
   - Identificación de transacciones canceladas
   - Detección de valores negativos o inconsistentes
   - Análisis de distribuciones extremas

3. **Corrección de inconsistencias**
   - Estandarización de formatos
   - Validación de reglas de negocio
   - Creación de variables derivadas básicas

### Metodología de Limpieza

El proceso de limpieza sigue un enfoque conservador que prioriza la preservación de información válida mientras elimina datos que puedan comprometer la calidad del análisis.

In [7]:
# Configuración inicial
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('default')
sns.set_palette("husl")

print("Librerías cargadas para preprocesamiento")
print(f"Fecha de procesamiento: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Librerías cargadas para preprocesamiento
Fecha de procesamiento: 2025-09-01 13:19:02


In [8]:
# Cargar datos del análisis exploratorio
try:
    df = pd.read_excel('../data/raw/online_retail.xlsx')
    print("Dataset original cargado desde archivo Excel")
except FileNotFoundError:
    print("Archivo no encontrado. Creando dataset sintético...")
    # Recrear dataset sintético consistente
    np.random.seed(42)
    n_samples = 10000
    
    df = pd.DataFrame({
        'InvoiceNo': [f'53{i:04d}' for i in range(n_samples)],
        'StockCode': np.random.choice(['85123A', '71053', '84406B'], n_samples),
        'Description': np.random.choice(['WHITE HANGING HEART', 'WHITE METAL LANTERN'], n_samples),
        'Quantity': np.random.poisson(3, n_samples) + 1,
        'InvoiceDate': pd.date_range('2010-12-01', '2011-12-09', periods=n_samples),
        'UnitPrice': np.random.gamma(2, 2, n_samples),
        'CustomerID': np.random.choice(range(12346, 18287), n_samples),
        'Country': np.random.choice(['United Kingdom', 'France', 'Germany'], n_samples, p=[0.7, 0.15, 0.15])
    })
    
    # Introducir problemas de calidad para demostrar limpieza
    missing_mask = np.random.random(n_samples) < 0.25
    df.loc[missing_mask, 'CustomerID'] = np.nan
    
    # Introducir cantidades negativas (devoluciones)
    negative_mask = np.random.random(n_samples) < 0.05
    df.loc[negative_mask, 'Quantity'] = -df.loc[negative_mask, 'Quantity']
    
    # Introducir facturas canceladas
    cancel_mask = np.random.random(n_samples) < 0.03
    df.loc[cancel_mask, 'InvoiceNo'] = 'C' + df.loc[cancel_mask, 'InvoiceNo']

# Convertir fecha a datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

print(f"Dataset inicial: {df.shape[0]:,} filas × {df.shape[1]} columnas")
print(f"Período de datos: {df['InvoiceDate'].min()} a {df['InvoiceDate'].max()}")

Dataset original cargado desde archivo Excel
Dataset inicial: 541,909 filas × 8 columnas
Período de datos: 2010-12-01 08:26:00 a 2011-12-09 12:50:00


## 1. Análisis Previo a la Limpieza

Antes de proceder con la limpieza, es fundamental entender la naturaleza y el impacto de los problemas de calidad identificados.

In [9]:
# Análisis detallado de problemas de calidad
print("ANÁLISIS DE PROBLEMAS DE CALIDAD")
print("=" * 40)

# 1. Valores faltantes en CustomerID
missing_customers = df['CustomerID'].isnull().sum()
missing_pct = (missing_customers / len(df)) * 100
print(f"1. CustomerID faltantes: {missing_customers:,} ({missing_pct:.1f}%)")

# 2. Transacciones canceladas
cancelled_transactions = df['InvoiceNo'].astype(str).str.startswith('C').sum()
cancelled_pct = (cancelled_transactions / len(df)) * 100
print(f"2. Transacciones canceladas: {cancelled_transactions:,} ({cancelled_pct:.1f}%)")

# 3. Cantidades negativas
negative_qty = (df['Quantity'] < 0).sum()
negative_qty_pct = (negative_qty / len(df)) * 100
print(f"3. Cantidades negativas: {negative_qty:,} ({negative_qty_pct:.1f}%)")

# 4. Cantidades cero
zero_qty = (df['Quantity'] == 0).sum()
zero_qty_pct = (zero_qty / len(df)) * 100
print(f"4. Cantidades cero: {zero_qty:,} ({zero_qty_pct:.1f}%)")

# 5. Precios negativos o cero
negative_price = (df['UnitPrice'] <= 0).sum()
negative_price_pct = (negative_price / len(df)) * 100
print(f"5. Precios negativos/cero: {negative_price:,} ({negative_price_pct:.1f}%)")

# Visualización de problemas de calidad
quality_issues = pd.DataFrame({
    'Problema': ['CustomerID Faltante', 'Transacciones Canceladas', 'Cantidades Negativas', 
                'Cantidades Cero', 'Precios Negativos/Cero'],
    'Cantidad': [missing_customers, cancelled_transactions, negative_qty, zero_qty, negative_price],
    'Porcentaje': [missing_pct, cancelled_pct, negative_qty_pct, zero_qty_pct, negative_price_pct]
})

fig = px.bar(
    quality_issues,
    x='Problema',
    y='Porcentaje',
    title='Distribución de Problemas de Calidad de Datos',
    labels={'Porcentaje': 'Porcentaje del Dataset (%)', 'Problema': 'Tipo de Problema'},
    text='Porcentaje'
)

fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(
    height=500,
    title_x=0.5,
    xaxis_tickangle=-45
)

fig.show()

ANÁLISIS DE PROBLEMAS DE CALIDAD
1. CustomerID faltantes: 135,080 (24.9%)
2. Transacciones canceladas: 9,288 (1.7%)
3. Cantidades negativas: 10,624 (2.0%)
4. Cantidades cero: 0 (0.0%)
5. Precios negativos/cero: 2,517 (0.5%)


## 2. Proceso de Limpieza de Datos

### Estrategia de Limpieza

La estrategia de limpieza se basa en los siguientes principios:

1. **Eliminación de registros sin CustomerID**: Estos registros no aportan valor para el análisis de fidelización
2. **Exclusión de transacciones canceladas**: Las cancelaciones no reflejan comportamiento de compra real
3. **Filtrado de cantidades y precios válidos**: Solo transacciones con valores positivos
4. **Preservación de la integridad temporal**: Mantener la secuencia temporal de las transacciones

In [10]:
# Proceso de limpieza paso a paso
print("PROCESO DE LIMPIEZA DE DATOS")
print("=" * 35)

# Crear copia para limpieza
df_clean = df.copy()
print(f"Dataset inicial: {df_clean.shape[0]:,} registros")

# Paso 1: Eliminar registros sin CustomerID
initial_count = len(df_clean)
df_clean = df_clean.dropna(subset=['CustomerID'])
removed_customers = initial_count - len(df_clean)
print(f"Paso 1 - Eliminados por CustomerID faltante: {removed_customers:,}")
print(f"         Registros restantes: {len(df_clean):,}")

# Paso 2: Eliminar transacciones canceladas
initial_count = len(df_clean)
df_clean = df_clean[~df_clean['InvoiceNo'].astype(str).str.startswith('C')]
removed_cancelled = initial_count - len(df_clean)
print(f"Paso 2 - Eliminadas por cancelación: {removed_cancelled:,}")
print(f"         Registros restantes: {len(df_clean):,}")

# Paso 3: Eliminar cantidades negativas o cero
initial_count = len(df_clean)
df_clean = df_clean[df_clean['Quantity'] > 0]
removed_quantity = initial_count - len(df_clean)
print(f"Paso 3 - Eliminadas por cantidad inválida: {removed_quantity:,}")
print(f"         Registros restantes: {len(df_clean):,}")

# Paso 4: Eliminar precios negativos o cero
initial_count = len(df_clean)
df_clean = df_clean[df_clean['UnitPrice'] > 0]
removed_price = initial_count - len(df_clean)
print(f"Paso 4 - Eliminadas por precio inválido: {removed_price:,}")
print(f"         Registros restantes: {len(df_clean):,}")

# Resumen de limpieza
total_removed = len(df) - len(df_clean)
retention_rate = (len(df_clean) / len(df)) * 100

print(f"\nRESUMEN DE LIMPIEZA:")
print(f"Registros originales: {len(df):,}")
print(f"Registros eliminados: {total_removed:,}")
print(f"Registros finales: {len(df_clean):,}")
print(f"Tasa de retención: {retention_rate:.1f}%")

PROCESO DE LIMPIEZA DE DATOS
Dataset inicial: 541,909 registros
Paso 1 - Eliminados por CustomerID faltante: 135,080
         Registros restantes: 406,829
Paso 2 - Eliminadas por cancelación: 8,905
         Registros restantes: 397,924
Paso 3 - Eliminadas por cantidad inválida: 0
         Registros restantes: 397,924
Paso 4 - Eliminadas por precio inválido: 40
         Registros restantes: 397,884

RESUMEN DE LIMPIEZA:
Registros originales: 541,909
Registros eliminados: 144,025
Registros finales: 397,884
Tasa de retención: 73.4%


## 3. Creación de Variables Derivadas

Una vez limpio el dataset, procedemos a crear variables derivadas que serán útiles para el análisis posterior.

In [11]:
# Creación de variables derivadas
print("CREACIÓN DE VARIABLES DERIVADAS")
print("=" * 40)

# 1. Variable de ingresos por transacción
df_clean['Revenue'] = df_clean['Quantity'] * df_clean['UnitPrice']
print("1. Variable 'Revenue' creada (Quantity × UnitPrice)")

# 2. Componentes temporales
df_clean['Year'] = df_clean['InvoiceDate'].dt.year
df_clean['Month'] = df_clean['InvoiceDate'].dt.month
df_clean['DayOfWeek'] = df_clean['InvoiceDate'].dt.dayofweek
df_clean['Hour'] = df_clean['InvoiceDate'].dt.hour
df_clean['Quarter'] = df_clean['InvoiceDate'].dt.quarter
print("2. Componentes temporales extraídos (Year, Month, DayOfWeek, Hour, Quarter)")

# 3. Variables categóricas derivadas
df_clean['IsWeekend'] = df_clean['DayOfWeek'].isin([5, 6])
df_clean['TimeOfDay'] = pd.cut(
    df_clean['Hour'], 
    bins=[0, 6, 12, 18, 24], 
    labels=['Madrugada', 'Mañana', 'Tarde', 'Noche'],
    include_lowest=True
)
print("3. Variables categóricas derivadas (IsWeekend, TimeOfDay)")

# Verificar las nuevas variables
print(f"\nDataset final: {df_clean.shape[0]:,} filas × {df_clean.shape[1]} columnas")
print(f"Nuevas columnas: {[col for col in df_clean.columns if col not in df.columns]}")

# Estadísticas básicas de la variable Revenue
print(f"\nEstadísticas de Revenue:")
print(f"Media: £{df_clean['Revenue'].mean():.2f}")
print(f"Mediana: £{df_clean['Revenue'].median():.2f}")
print(f"Desviación estándar: £{df_clean['Revenue'].std():.2f}")
print(f"Rango: £{df_clean['Revenue'].min():.2f} - £{df_clean['Revenue'].max():.2f}")

CREACIÓN DE VARIABLES DERIVADAS
1. Variable 'Revenue' creada (Quantity × UnitPrice)
2. Componentes temporales extraídos (Year, Month, DayOfWeek, Hour, Quarter)
3. Variables categóricas derivadas (IsWeekend, TimeOfDay)

Dataset final: 397,884 filas × 16 columnas
Nuevas columnas: ['Revenue', 'Year', 'Month', 'DayOfWeek', 'Hour', 'Quarter', 'IsWeekend', 'TimeOfDay']

Estadísticas de Revenue:
Media: £22.40
Mediana: £11.80
Desviación estándar: £309.07
Rango: £0.00 - £168469.60


## 4. Análisis Post-Limpieza

Evaluamos el impacto de la limpieza en las características del dataset.

In [12]:
# Análisis comparativo antes y después de la limpieza
print("ANÁLISIS POST-LIMPIEZA")
print("=" * 30)

# Comparación de estadísticas básicas
comparison_stats = pd.DataFrame({
    'Métrica': ['Registros totales', 'Clientes únicos', 'Productos únicos', 'Facturas únicas'],
    'Original': [
        len(df),
        df['CustomerID'].nunique(),
        df['StockCode'].nunique(),
        df['InvoiceNo'].nunique()
    ],
    'Limpio': [
        len(df_clean),
        df_clean['CustomerID'].nunique(),
        df_clean['StockCode'].nunique(),
        df_clean['InvoiceNo'].nunique()
    ]
})

comparison_stats['Cambio_%'] = (
    (comparison_stats['Limpio'] - comparison_stats['Original']) / 
    comparison_stats['Original'] * 100
).round(1)

print("Comparación antes y después de la limpieza:")
display(comparison_stats)

# Visualización de distribuciones
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Distribución de Quantity', 'Distribución de UnitPrice', 
                   'Distribución de Revenue', 'Transacciones por Mes'),
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}]]
)

# Quantity distribution
fig.add_trace(
    go.Histogram(x=df_clean['Quantity'], name='Quantity', nbinsx=50),
    row=1, col=1
)

# UnitPrice distribution
fig.add_trace(
    go.Histogram(x=df_clean['UnitPrice'], name='UnitPrice', nbinsx=50),
    row=1, col=2
)

# Revenue distribution
fig.add_trace(
    go.Histogram(x=df_clean['Revenue'], name='Revenue', nbinsx=50),
    row=2, col=1
)

# Monthly transactions
monthly_transactions = df_clean.groupby(['Year', 'Month']).size().reset_index(name='Transactions')
monthly_transactions['YearMonth'] = monthly_transactions['Year'].astype(str) + '-' + monthly_transactions['Month'].astype(str).str.zfill(2)

fig.add_trace(
    go.Scatter(x=monthly_transactions['YearMonth'], y=monthly_transactions['Transactions'], 
              mode='lines+markers', name='Transacciones Mensuales'),
    row=2, col=2
)

fig.update_layout(height=800, title_text="Análisis de Distribuciones Post-Limpieza")
fig.show()

ANÁLISIS POST-LIMPIEZA
Comparación antes y después de la limpieza:


,Métrica,Original,Limpio,Cambio_%
0,Registros totales,541909,397884,-26.6
1,Clientes únicos,4372,4338,-0.8
2,Productos únicos,4070,3665,-10.0
3,Facturas únicas,25900,18532,-28.4


NameError: name 'make_subplots' is not defined

## 5. Guardado del Dataset Limpio

Finalmente, guardamos el dataset limpio para su uso en las siguientes fases del proyecto.

In [13]:
# Guardar dataset limpio
output_path = '../data/processed/online_retail_clean.csv'
df_clean.to_csv(output_path, index=False)

print("DATASET LIMPIO GUARDADO")
print("=" * 30)
print(f"Archivo: {output_path}")
print(f"Dimensiones finales: {df_clean.shape[0]:,} filas × {df_clean.shape[1]} columnas")
print(f"Período de datos: {df_clean['InvoiceDate'].min()} a {df_clean['InvoiceDate'].max()}")
print(f"Clientes únicos: {df_clean['CustomerID'].nunique():,}")
print(f"Productos únicos: {df_clean['StockCode'].nunique():,}")
print(f"Revenue total: £{df_clean['Revenue'].sum():,.2f}")

# Resumen de calidad final
print(f"\nCALIDAD DEL DATASET FINAL:")
print(f"Valores faltantes: {df_clean.isnull().sum().sum()}")
print(f"Duplicados: {df_clean.duplicated().sum()}")
print(f"Registros con valores negativos: 0")
print(f"Tasa de retención: {len(df_clean)/len(df)*100:.1f}%")

print("\nDataset listo para la fase de ingeniería de características.")

DATASET LIMPIO GUARDADO
Archivo: ../data/processed/online_retail_clean.csv
Dimensiones finales: 397,884 filas × 16 columnas
Período de datos: 2010-12-01 08:26:00 a 2011-12-09 12:50:00
Clientes únicos: 4,338
Productos únicos: 3,665
Revenue total: £8,911,407.90

CALIDAD DEL DATASET FINAL:
Valores faltantes: 0
Duplicados: 5192
Registros con valores negativos: 0
Tasa de retención: 73.4%

Dataset listo para la fase de ingeniería de características.
